In [2]:
with open("inputs/day23.txt", "r") as fn:
    input = fn.read()

lines = input.splitlines() 

# Part 1

In [3]:
dirs = ((0,1),(1,0),(-1,0),(0,-1))

slope_map = {
    (0,1): ">",
    (1,0): "v",
    (-1,0): "^",
    (0,-1): "<"
}

solutions = list()
def solve(path):
    global global_solutions
    if path[-1]==exit:
        solutions.append(path)    
    
    for dir in dirs:
        step = (path[-1][0] + dir[0], path[-1][1] + dir[1])

        in_bounds = (0 < step[0] < len(lines)) and (0 < step[1] < len(lines[0]))
        is_path = lines[step[0]][step[1]] in (".", slope_map[dir]) if in_bounds else False
        not_visited = step not in path
        if all((in_bounds, is_path, not_visited)):
            solve(path + [step])



In [4]:
entrance = (0,1)
exit = (len(lines)-1, len(lines[0])-2)
solutions = list()

solve([entrance])

max([len(s) for s in solutions]) -1

2018

# Part 2 - Networkx

In [5]:
import numpy as np
import networkx as nx

grid = np.genfromtxt("inputs/day23.txt", delimiter=1, dtype=str, comments=None)

In [6]:
G = nx.Graph()

for i, x in np.ndenumerate(grid):
    if x != "#":
        G.add_node(i)

        for dir in ((0,1),(1,0),(-1,0),(0,-1)):
            step = (i[0] + dir[0], i[1] + dir[1]) 
            
            in_bounds = (0 < step[0] < grid.shape[0]) and (0 < step[1] < grid.shape[1])
            is_path = grid[*step] in (".", "<", ">", "^", "v") if in_bounds else False
            if in_bounds and is_path:
                G.add_edge(i, step, weight=1)


# Simplfy graph
while True:
    changed = False
    for node in G.nodes:
        n = list(nx.neighbors(G, node))
        if len(n) ==2:
            G.add_edge(*n, weight=sum([e[2]['weight'] for e in list(G.edges(node, data=True))]))
            G.remove_node(node)
            changed = True
            break
    if not changed:
        break

In [7]:
entrance = (0,1)
exit = (grid.shape[0]-1, grid.shape[1]-2)

_max = 0 
for p in nx.all_simple_paths(G, entrance, exit):
    _max = max(_max, nx.path_weight(G, p, weight='weight'))
_max

6406